In [1]:
import os
import json

import cv2
import keras
from keras import backend as K
from keras.models import Model
from keras.layers import Input
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.losses import binary_crossentropy
from keras.callbacks import Callback, ModelCheckpoint
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

In [3]:
tf.test.gpu_device_name()
dir = "/content/drive/My Drive/Steel-Defect-Detection-CNN/"

## PREPROCESSING

In [4]:
sub_df = pd.read_csv(dir+'submission_thirat.csv')
sub_df['ImageId'] = sub_df['ImageId_ClassId'].apply(lambda x: x.split('_')[0])
sub_df['ClassId'] = sub_df['ImageId_ClassId'].apply(lambda x: x.split('_')[1])
test_imgs = pd.DataFrame(sub_df['ImageId'].unique(), columns=['ImageId'])

## UTILITY

In [5]:
def mask2rle(img):
    """
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    """
    pixels = img.T.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)


def rle2mask(mask_rle, shape=(256, 1600)):
    """
    mask_rle: run-length as string formated (start length)
    shape: (width,height) of array to return
    Returns numpy array, 1 - mask, 0 - background

    """
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0] * shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape).T

In [6]:
def build_masks(rles, input_shape):
    depth = len(rles)
    height, width = input_shape
    masks = np.zeros((height, width, depth))
    for i, rle in enumerate(rles):
        print(i, rle)
        if type(rle) is str:
            masks[:, :, i] = rle2mask(rle, (width, height))
    return masks


def build_rles(masks):
    width, height, depth = masks.shape

    rles = [mask2rle(masks[:, :, i])
            for i in range(depth)]

    return rles

In [7]:
def dice_coef(y_true, y_pred, smooth=1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)


def dice_loss(y_true, y_pred):
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = y_true_f * y_pred_f
    score = (2. * K.sum(intersection) + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    return 1. - score


def bce_dice_loss(y_true, y_pred):
    return binary_crossentropy(y_true, y_pred) + dice_loss(y_true, y_pred)


# >>thirat

def combine_rles(sample_rles, sample_class):
    combine_sample_rles = []
    j = 0
    for i in range(4):
        if i + 1 in sample_class:  # and j==0:
            combine_sample_rles.append(sample_rles[j])
            j += 1
        else:
            combine_sample_rles.append("")
    return combine_sample_rles

## Sample

## Data Generation

In [8]:
class DataGenerator(keras.utils.Sequence):
    """Generates data for Keras"""

    def __init__(self, list_IDs, df, target_df=None, mode='fit',
                 base_path='train_images/',
                 batch_size=1, dim=(256, 1600), n_channels=1,
                 n_classes=4, random_state=2019, shuffle=True):
        self.dim = dim
        self.batch_size = batch_size  # 32
        self.df = df
        self.mode = mode
        self.base_path = base_path
        self.target_df = target_df
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.random_state = random_state

        self.on_epoch_end()

    def __len__(self):
        """Denotes the number of batches per epoch"""
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        """Generate one batch of data"""
        # Generate indexes of the batch
        indexes = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]

        # Find list of IDs
        list_IDs_batch = [self.list_IDs[k] for k in indexes]

        X = self.__generate_X(list_IDs_batch)
        X = X.astype(float)
        if self.mode == 'fit':
            y = self.__generate_y(list_IDs_batch)
            y = y.astype(float)
            return X, y

        elif self.mode == 'predict':
            return X

        else:
            raise AttributeError('The mode parameter should be set to "fit" or "predict".')

    def on_epoch_end(self):
        """Updates indexes after each epoch"""
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.seed(self.random_state)
            np.random.shuffle(self.indexes)

    def __generate_X(self, list_IDs_batch):
        """Generates data containing batch_size samples"""
        # Initialization
        X = np.empty((self.batch_size, *self.dim, self.n_channels))

        # Generate data
        for i, ID in enumerate(list_IDs_batch):
            im_name = self.df['ImageId'].iloc[ID]
            img_path = f"{self.base_path}/{im_name}"
            img = self.__load_grayscale(img_path)

            # Store samples
            X[i,] = img

        return X

    def __generate_y(self, list_IDs_batch):
        y = np.empty((self.batch_size, *self.dim, self.n_classes), dtype=int)
        print(self.batch_size, *self.dim, self.n_classes)
        for i, ID in enumerate(list_IDs_batch):
            im_name = self.df['ImageId'].iloc[ID]
            print(im_name)
            image_df = self.target_df[self.target_df['ImageId'] == im_name]
            sample_class = image_df['ClassId'].values
            sample_rles = image_df['EncodedPixels'].values
            combine_sample_rles = combine_rles(sample_rles, sample_class)
            masks = build_masks(combine_sample_rles, input_shape=self.dim)

            y[i,] = masks

        return y

    def __load_grayscale(self, img_path):
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        img = img.astype(np.float32) / 255.
        img = np.expand_dims(img, axis=-1)

        return img

    def __load_rgb(self, img_path):
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = img.astype(np.float32) / 255.

        return img

In [9]:
BATCH_SIZE = 16

In [10]:
def build_model(input_shape):
    inputs = Input(input_shape)

    c1 = Conv2D(8, (3, 3), activation='relu', padding='same')(inputs)
    c1 = Conv2D(8, (3, 3), activation='relu', padding='same')(c1)
    p1 = MaxPooling2D((2, 2))(c1)

    c2 = Conv2D(16, (3, 3), activation='relu', padding='same')(p1)
    c2 = Conv2D(16, (3, 3), activation='relu', padding='same')(c2)
    p2 = MaxPooling2D((2, 2))(c2)

    c3 = Conv2D(32, (3, 3), activation='relu', padding='same')(p2)
    c3 = Conv2D(32, (3, 3), activation='relu', padding='same')(c3)
    p3 = MaxPooling2D((2, 2))(c3)

    c4 = Conv2D(64, (3, 3), activation='relu', padding='same')(p3)
    c4 = Conv2D(64, (3, 3), activation='relu', padding='same')(c4)
    p4 = MaxPooling2D(pool_size=(2, 2))(c4)

    c5 = Conv2D(64, (3, 3), activation='relu', padding='same')(p4)
    c5 = Conv2D(64, (3, 3), activation='relu', padding='same')(c5)
    p5 = MaxPooling2D(pool_size=(2, 2))(c5)

    c55 = Conv2D(128, (3, 3), activation='relu', padding='same')(p5)
    c55 = Conv2D(128, (3, 3), activation='relu', padding='same')(c55)

    u6 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c55)
    u6 = concatenate([u6, c5])
    c6 = Conv2D(64, (3, 3), activation='relu', padding='same')(u6)
    c6 = Conv2D(64, (3, 3), activation='relu', padding='same')(c6)

    u71 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c6)
    u71 = concatenate([u71, c4])
    c71 = Conv2D(32, (3, 3), activation='relu', padding='same')(u71)
    c61 = Conv2D(32, (3, 3), activation='relu', padding='same')(c71)

    u7 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c61)
    u7 = concatenate([u7, c3])
    c7 = Conv2D(32, (3, 3), activation='relu', padding='same')(u7)
    c7 = Conv2D(32, (3, 3), activation='relu', padding='same')(c7)

    u8 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c7)
    u8 = concatenate([u8, c2])
    c8 = Conv2D(16, (3, 3), activation='relu', padding='same')(u8)
    c8 = Conv2D(16, (3, 3), activation='relu', padding='same')(c8)

    u9 = Conv2DTranspose(8, (2, 2), strides=(2, 2), padding='same')(c8)
    u9 = concatenate([u9, c1], axis=3)
    c9 = Conv2D(8, (3, 3), activation='relu', padding='same')(u9)
    c9 = Conv2D(8, (3, 3), activation='relu', padding='same')(c9)

    outputs = Conv2D(4, (1, 1), activation='sigmoid')(c9)

    model = Model(inputs=[inputs], outputs=[outputs])
    model.compile(optimizer='adam', loss=bce_dice_loss, metrics=[dice_coef])

    return model

In [11]:
model = build_model((256, 1600, 1))
model.summary()
model.load_weights(filepath=dir+'model.h5')
test_df = []

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 1600, 1 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 256, 1600, 8) 80          input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 256, 1600, 8) 584         conv2d[0][0]                     
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 128, 800, 8)  0           conv2d_1[0][0]                   
_______________________________________________________________________________________

In [ ]:
for i in range(0, test_imgs.shape[0], 500):
    batch_idx = list(
        range(i, min(test_imgs.shape[0], i + 500))
    )
    
    test_generator = DataGenerator(
        batch_idx,
        df=test_imgs,
        shuffle=False,
        mode='predict',
        base_path=dir+'test_images',
        target_df=sub_df,
        batch_size=1,
        n_classes=4
    )
    
    batch_pred_masks = model.predict_generator(
        test_generator, 
        workers=1,
        verbose=1,
        use_multiprocessing=False
    )

Instructions for updating:
Please use Model.predict, which supports generators.
500/500 [==============================] - 7s 14ms/step


In [ ]:

for j, b in tqdm(enumerate(batch_idx)):
    filename = test_imgs['ImageId'].iloc[b]
    pred_masks = batch_pred_masks[j,].round().astype(int)
    for k in range(1,5):
        image_df = sub_df[(sub_df['ImageId'] == filename) & (sub_df['ClassId'] == k)].copy()
        pred_rles = build_rles(pred_masks)
        image_df['EncodedPixels'] = pred_rles[k-1]
        test_df.append(image_df)

In [ ]:
print(test_df)

In [ ]:
test_df = list(test_df)

In [ ]:
test_df = pd.concat(test_df)
test_df.to_csv(dir+'predictions.csv', index=False)

In [ ]:
print(test_imgs)

In [ ]:
predict_csv = pd.read_csv(dir+'predictions.csv')

In [ ]:
def check_result(file):
    sample_filename = file
    sample_image_df = predict_csv[predict_csv['ImageId'] == sample_filename]

    sample_path = f"test_images/{sample_image_df['ImageId'].iloc[0]}"
    sample_img = cv2.imread(sample_path)
    print(sample_path)
    sample_rles = sample_image_df['EncodedPixels'].values
    sample_class = sample_image_df['ClassId'].values
    combine_sample_rles = combine_rles(sample_rles, sample_class)
    sample_masks = build_masks(combine_sample_rles, input_shape=(256, 1600))
    
    fig, axs = plt.subplots(5, figsize=(12, 12))
    axs[0].imshow(sample_img)
    axs[0].axis('off')
    for i in range(4):
        # print(sample_masks[:, :, 0],len(sample_masks[:, :, 0]))
        # print(sample_masks, len(sample_masks))
        axs[i + 1].imshow(sample_masks[:, :, i])
        axs[i + 1].axis('off')
    plt.show()

In [ ]:
check_result("0000f269f.jpg")

In [ ]:
check_result("0042e163f.jpg")

In [ ]:
check_result("17a6dce92.jpg")